In [4]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from pymongo import MongoClient

In [3]:
gl_cat_tfidf = gl.SFrame('../data/items_categories_tifidf_need2reduce.gl')
train_set_2 = gl.SFrame("../data/train_set_2.gl")
test_set_2 = gl.SFrame("../data/test_set_2.gl")
orig_results = gl.SFrame("../data/orig_results.gl")

In [73]:
client = MongoClient()
# Access/Initiate Database
db = client['Podcast']
# Access/Initiate Table
Features = db['Features']
print Features.count()

keep_cols = []
for i in Features.find({}):
    keep_cols.append(i['Feature'])

413


In [78]:
other_model = gl.load_model("../data/ItemSimilarity.gl")

In [81]:
results = other_model.get_similar_items(["Car Talk"])
results['similar']

Getting similar items completed in 0.016432

In [83]:
results['similar']

dtype: str
Rows: 10
['Fast Lane Daily', 'Sports Car Unleashed', 'SEMA 2014', 'Ask Me Another', 'Velocity', 'Mixing The Vibe', 'Opposite Lock', 'Omar Kelly Show', 'Motorz', 'This Week Inside Sim Racing']

In [76]:
%%writefile get_best_features_OneInOut.py

import graphlab as gl
gl.canvas.set_target('ipynb')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from pymongo import MongoClient

gl_cat_tfidf = gl.SFrame('../data/items_categories_tifidf_need2reduce.gl')
train_set_2 = gl.SFrame("../data/train_set_2.gl")
test_set_2 = gl.SFrame("../data/test_set_2.gl")
orig_res = gl.SFrame("../data/orig_results.gl")

client = MongoClient()
# Access/Initiate Database
db = client['Podcast']
# Access/Initiate Table
Features = db['Features_noCategories']
print Features.count()

precision = []
prec_num  = []
recall    = []
recall_num= []

for col in gl_cat_tfidf.column_names()[:-1]:
    cols = ['Title']
    if col == 'Title':
        continue
    cols.append(col)
    item_model = gl.item_similarity_recommender.create(train_set_2, user_id="user_id", 
                                                       item_id="Title", item_data=gl_cat_tfidf[cols],
                                                       verbose=False)
    new_res = item_model.evaluate_precision_recall(test_set_2, cutoffs=[1,2,3,4,5], verbose=False)
    new_res = new_res['precision_recall_overall']
    compare_p = sum(orig_res['precision']<new_res['precision'])
    compare_r = sum(orig_res['recall']<new_res['recall'])
    if compare_p>0 or compare_r>0:
        p_max = np.array(orig_res['precision']- new_res['precision']).max()
        r_max = np.array(orig_res['recall']- new_res['recall']).max()
        Features.insert_one( {'Feature': col, 'Precision': compare_p, 
                      'Recall': compare_r, 'Precision_max': p_max, 
                      'Recall_max':r_max})
        print "Precision: " + str(col)
        print "Recall: " + str(col)

Writing get_best_features_OneInOut.py


In [37]:
#new_res = item_model.evaluate_precision_recall(test_set_2, cutoffs=[1,2,3,4,5], verbose=False)
#new_res = new_res['precision_recall_overall']
compare_p = sum(orig_res['precision']<new_res['precision'])
compare_r = sum(orig_res['recall']<new_res['recall'])

p_max = np.array(orig_res['precision']- new_res['precision']).max()
r_max = np.array(orig_res['recall']- new_res['recall']).max()

In [36]:
np.array(orig_res['precision'] - new_res['precision']).max()

0.00012853896804282394

In [6]:
a = np.array([0,5,10,15])
b = np.array([1,5.5,12,17])

In [16]:
a = True
b = False

In [17]:
if a is True or b is True:
    print "hola"

hola


In [ ]:
Podcasts.insert_one( {'Feature': col, 'Precision': compare_p, 
                      'Recall': compare_r, 'Precision_max': p_max, 
                      'Recall_max':r_max})